# The Architecture format
A complete FPGA is specified in an architecture XML file. The FPGA specification is wrapped in an `<architecture>` tag; `v8_example_arch.xml` defines a simple FPGA.

### [`<models>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#recognized-blif-models-models)
The first element in the architecture is `<models>`, which describes `blif` circuit models that the FPGA uses. The models  `.names`, `.latch`, `.input`, and `.output` are automatically recognized and don't need to be specified in this section. The `v8_example_arch.xml` architecture is simple enough to not need any additional models specified here.


### [`<layout>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#fpga-grid-layout)
The `<layout>` element specifies how the FPGA grid will be arranged. In `v8_example_arch.xml`: `<layout auto="1.000000"/>` specifies an automatic grid with an aspect ratio of 1.0. A specific width and heigh could be specified instead of the automatic layout. The rest of the `<layout>` section specifies that the perimeter will have `io` blocks, the corners will be `EMPTY` and the rest of the FPGA will be filled with `clb` blocks. Higher number priorities take precedence over lower numbers, so the `EMPTY` blocks in the corners will take precedence over the `io` blocks around the perimeter. The lowest priority `clb` blocks fill the remaining places in the grid.

`io` and `clb` blocks are defined in the `<complexblocklist>` section.

### [`<device>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#fpga-device-information)
The `<device>` element characterizes the transistors and connections of the FPGA. `<sizing>` specifies the resistance of the minimum-width nmos and pmos transistors. `<area grid_logic_tile_area>` is used as an estimate of the size of one grid tile.

The `<chan_width_distr>` section sets the relative widths of the routing channels in various parts of the FPGA. Here, all channels are set to be distributed uniformly. 

`<switch_block type="wilton" fs="3"/>` indicates a [Wilton switch block pattern](http://docs.verilogtorouting.org/en/latest/references/#wilton-phd) with a  switchblock fraction (fs) of 3.

`<connection_block input_switch_name="ipin_cblock"/>` specifies the name of the connection block. The actual connection block is defined in the `<switchlist>` section.

### [`<switchlist>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#switches)
The `<switchlist>` section specifies the switches used to connect wires and pins together. The switch names defined here are used elsewhere in the `<connection_block>` and `<segmentlist>` sections in this example architecture.


### [`<segmentlist>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#wire-segments)
`<segmentlist>` defines the wire segments in the channels between logic blocks. The `<segment>` element specifies one kind of wire to be used. `freq` is used to indicate the the usage ratio of this kind of wire with respect to all the wires. `length` is how many logic blocks the wire segment will span. `Rmetal` and `Cmetal` indicate the resistance and capacitance of the segment.

`<mux name>` is the name of the switch used to drive the segment and is defined in the `<switchlist>` section.

`<sb type>` is the pattern of switch blocks on the segment, and because switch blocks are between logic blocks there are one more entries in the list than the length of the segment. So for the segment with a length of 4, there are 5 entries in the pattern. `<cb type>` is the pattern of connection blocks on the segment. Connection blocks are next to logic blocks so the number of entries in the pattern equals the length of the segment.


## [`<complexblocklist>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#complex-blocks)
In this example architecture, two complex blocks are defined: An input/output block, and a configurable logic block (CLB).

### Input and Output block
`<pb_type name="io" capacity="3">` begins the definition of the I/O block. `pb_type` is the tag that indicates a primitive block is being defined, and multiple `pb_type`'s can be nested. `capacity` is only allowed in an I/O block, and indicates how many inputs or outpus the block will have.

`<input name="outpad" num_pins="1"/>`  defines what might be considered an *output* from the FPGA, which is why it is given the name `outpad`: signals coming out of the FPGA are routed to the *input* of this block and then sent out of the *outpad* to the outside world. Similarly, a signal from the outside world comes in the *inpad* and then sent out the *output* of the block to then be routed through the FPGA. Thus the *input* to the FPGA is defined by `<output name="inpad" num_pins="1"/>`. 

An I/O block can only be an input or an output, and the mode defines the behavior. `<mode name="inpad">` begins the input behavior. `<pb_type name="inpad" blif_model=".input" num_pb="1">` gives the physical block a name `inpad`, associates it with the `.input` of the blif model, and indicates there is 1 of these `inpad`s in the parent pb_type of `io`.

`<output name="inpad" num_pins="1"/>` says that there is one output of the block named `inpad`. The signal comes in the *inpad* and is sent *out* of this block to the rest of the FPGA.

The `<interconnect>` section makes connections between the block's pins. `<direct name="inpad" input="inpad.inpad" output="io.inpad">` creates a direct connection named `inpad`, connects its input to `inpad.inpad` which is the output of the `inpad` block, and connects its ouput to `io.inpad` which is the ouput of the `io` block.

The `<delay_constant>` tag specifies the delay between the in_port and the out_port of the primitive block.

`<mode name="outpad">` defines the other mode of this block, and is analagous to the `inpad` definition.



## Logic Blocks

`<pb_type name="clb">` is where the general purpuse *complex logic block* or *clb* definition begins. `<pb_type>` is the tag to define a primitive block on the FPGA. Next, the inputs and outputs to the block are defined; here there are 10 inputs, 3 outputs, and 1 clock input. The inputs and outputs have `equivalent=true` attributes, which means that they are logically equivalent and so order doesn't matter when routing.

`<pb_type name="ble4" num_pb="3">` defines the *basic logic element* or *BLE* that makes up the clb. The `<pb_type name="ble4" num_pb="3">` attribute indicates that 3 of these BLEs called `ble4` are contained in the surrounding `clb`. The next lines define 4 inputs, one output, and one clock line.

`<pb_type name="lut4" blif_model=".names" num_pb="1" class="lut">` defines the lookup-table named `lut4`; `.names` is the BLIF keyword for lookup-table. The next two lines show 4 inputs and 1 output to the LUT. The `<delay_matrix>` attribute specifies the propogation delay through the LUT's inputs to output.

`<pb_type name="ff" blif_model=".latch" num_pb="1" class="flipflop">` defines the flip-flop in the BLE: 1 flip-flop named `ff`; `.latch` is the BLIF keyword for flip-flop. The next lines specify the I/Os and timing parameters.

The `<interconnect>` section wires the blocks together. The input of `ble4` is connected to the input of `lut4`, the output of `lut4` is connected to the D-input of `ff`. The clk inputs are wired together. Finally, a mux is defined to set the output of `ble4` to be either the direct output of `lut4` or the latched output of `ff`.

That completes the *ble* definition, now the *clb* needs to be completed with its `<interconnect>` section. The `<complete>` tag indicates a fully connected crossbar, so any input of the clb can be routed to any of the bles. Also, `input="clb.I ble4[2:0].out"` means that there are two inputs to the crossbar: the input to the clb as well as the output of the bles, giving a feedback configuration. The output of the crossbar is connected to the inputs of the bles. Finally, the output of the bles are connected to the output of the clb.

At the end of the top-level `pb_type` definition, the configuration of pins is specified. `<fc in_type="frac" in_val="0.15" out_type="frac" out_val="0.25"/>` shows the percentage of tracks in the channel drive the block's pins. 

`<pinlocations pattern="spread"/>` says that the pins should be spread evenly around the edges of the block.